# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd

In [7]:
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [8]:
df = pd.read_csv('previsao_de_renda.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [10]:
#1

df.drop(['Unnamed: 0', 'data_ref', 'index', 'mau'], axis = 1, inplace = True)

df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,1,Assalariado,Secundário,Casado,Casa,36,3.575342,3.0,3369.24
1,M,True,True,0,Empresário,Secundário,Casado,Casa,42,0.860274,2.0,6096.14
2,M,True,True,2,Assalariado,Superior completo,Casado,Casa,31,8.065753,4.0,5658.98
3,F,True,False,0,Empresário,Secundário,Casado,Casa,50,1.208219,2.0,7246.69
4,M,False,False,0,Assalariado,Secundário,Casado,Casa,52,13.873973,2.0,4017.37


In [11]:
print(df['tipo_renda'].value_counts())
print(df['educacao'].value_counts())
print(df['estado_civil'].value_counts())
print(df['tipo_residencia'].value_counts())

Assalariado         7826
Empresário          3311
Pensionista         2543
Servidor público    1316
Bolsista               4
Name: tipo_renda, dtype: int64
Secundário             8927
Superior completo      5262
Superior incompleto     599
Primário                193
Pós graduação            19
Name: educacao, dtype: int64
Casado      10550
Solteiro     1767
União        1168
Separado      838
Viúvo         677
Name: estado_civil, dtype: int64
Casa             13566
Com os pais        637
Governamental      443
Aluguel            184
Estúdio            107
Comunitário         63
Name: tipo_residencia, dtype: int64


In [20]:
y_a, X_a = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_veiculo) 
                    + C(posse_de_imovel)
                    + qtd_filhos 
                    + C(tipo_renda) 
                    + C(educacao, Treatment(2)) 
                    + C(estado_civil) 
                    + C(tipo_residencia, Treatment(1)) 
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia 
                    + 1''', df)

Modelo_A = sm.OLS(y_a, X_a).fit()
Modelo_A.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     165.3
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        11:16:40   Log-Likelihood:                -13204.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12441   BIC:                         2.664e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             7.2393      0.258     28.084      0.000       6.734       7.745
C(sexo)[T.M]                                          0.4813      0.014     33.673      0.000       0.453       0.509
C(posse_de_veiculo)[T.True]                          -0.0012      0.014     -0.087      0.931      -0.028       0.026
C(posse_de_imovel)[T.True]                            0.0849      0.014      6.275      0.000       0.058       0.111
C(tipo_renda)[T.Bolsista]                            -0.0493      0.350     -0.141      0.888      -0.735       0.636
C(tipo_renda)[T.Empresário]                           0.1786      0.015     12.209      0.000       0.150       0.207
C(tipo_renda)[T.Pensionista]                          0.3784      0.233      1.623      0.105      -0.079       0.835
C(tipo_renda)[T.Servidor público]                     0.0950      0.021      4.485      0.000       0.053       0.136
C(educacao, Treatment(2))[T.Primário]                -0.0164      0.071     -0.232      0.817      -0.156       0.123
C(educacao, Treatment(2))[T.Pós graduação]            0.5338      0.161      3.321      0.001       0.219       0.849
C(educacao, Treatment(2))[T.Superior completo]        0.0968      0.013      7.193      0.000       0.070       0.123
C(educacao, Treatment(2))[T.Superior incompleto]     -0.0261      0.031     -0.845      0.398      -0.087       0.034
C(estado_civil)[T.Separado]                          -0.0437      0.131     -0.334      0.738      -0.300       0.213
C(estado_civil)[T.Solteiro]                           0.0169      0.128      0.131      0.895      -0.235       0.269
C(estado_civil)[T.União]                             -0.0259      0.023     -1.110      0.267      -0.072       0.020
C(estado_civil)[T.Viúvo]                             -0.0422      0.134     -0.314      0.753      -0.306       0.221
C(tipo_residencia, Treatment(1))[T.Aluguel]          -0.0148      0.053     -0.278      0.781      -0.119       0.090
C(tipo_residencia, Treatment(1))[T.Com os pais]      -0.0720      0.029     -2.449      0.014      -0.130      -0.014
C(tipo_residencia, Treatment(1))[T.Comunitário]       0.0427      0.089      0.483      0.629      -0.131       0.216
C(tipo_residencia, Treatment(1))[T.Estúdio]          -0.1242      0.072     -1.732      0.083      -0.265       0.016
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0711      0.038     -1.893      0.058      -0.145       0.003
qtd_filhos                                            0.0087      0.128      0.068      0.946      -0.242       0.260
idade                                                 0.0052      0.001      6.625      0.000       0.004       0.007
tempo_emprego  

In [28]:
#2  Remova a variável menos significante e analise:
        #Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
        #Observe os parâmetros e veja se algum se alterou muito.
        
        #Decidi retirar 'estado_civil'
        
y_b, X_b = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_veiculo) 
                    + C(posse_de_imovel)
                    + qtd_filhos 
                    + C(tipo_renda) 
                    + C(educacao, Treatment(2)) 
                    + C(tipo_residencia, Treatment(1)) 
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia 
                    + 1''', df)

Modelo_B = sm.OLS(y_a, X_a).fit()
Modelo_B.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.241
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     208.1
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:41:00   Log-Likelihood:                -13210.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12446   BIC:                         2.661e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.2182      0.258     28.009      0.000       6.713       7.723
C(sexo)[T.M]                                         0.4802      0.014     33.700      0.000       0.452       0.508
C(posse_de_veiculo)[T.True]                         -0.0016      0.014     -0.116      0.908      -0.029       0.025
C(posse_de_imovel)[T.True]                           0.0937      0.013      7.075      0.000       0.068       0.120
C(tipo_renda)[T.Bolsista]                           -0.0498      0.350     -0.142      0.887      -0.735       0.636
C(tipo_renda)[T.Empresário]                          0.1787      0.015     12.220      0.000       0.150       0.207
C(tipo_renda)[T.Pensionista]                         0.3863      0.233      1.656      0.098      -0.071       0.843
C(tipo_renda)[T.Servidor público]                    0.0922      0.021      4.361      0.000       0.051       0.134
C(educacao, Treatment(2))[T.Primário]               -0.0203      0.071     -0.286      0.775      -0.159       0.119
C(educacao, Treatment(2))[T.Pós graduação]           0.5378      0.161      3.345      0.001       0.223       0.853
C(educacao, Treatment(2))[T.Superior completo]       0.0972      0.013      7.234      0.000       0.071       0.124
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0262      0.031     -0.848      0.396      -0.087       0.034
C(estado_civil)[T.Separado]                         -0.0451      0.131     -0.345      0.730      -0.302       0.211
C(estado_civil)[T.Solteiro]                          0.0141      0.128      0.110      0.912      -0.238       0.266
C(estado_civil)[T.União]                            -0.0273      0.023     -1.172      0.241      -0.073       0.018
C(estado_civil)[T.Viúvo]                            -0.0421      0.134     -0.313      0.754      -0.305       0.221
qtd_filhos                                           0.0083      0.128      0.065      0.948      -0.243       0.259
idade                                                0.0054      0.001      6.981      0.000       0.004       0.007
tempo_emprego                                        0.0483      0.001     48.463      0.000       0.046       0.050
qt_pessoas_residencia                                0.0077      0.128      0.060      0.952      -0.243       0.258
==============================================================================
Omnibus:                        1.073   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.585   Jarque-Bera (JB):                1.042
Skew:                          -0.012   Prob(JB):                        0.594
Kurtosis:                       3.038   Cond. No.                     2.62e+

A variável 'estado_civil' foi retirada porque mostrava os valores de P >0.05 em todos os exemplos. Entretanto, o valor de R² teve uma alteração pouco significativa, inclusive tento o valor de R² - ajustado igual.

**#1 Avalie os parâmetros e veja se parecem fazer sentido prático.**

Aumento da renda com P-Value < 0.05:

Coef positivo:
- posse_de_imovel True
- tipo_renda = empresário ou servidor público
- educacao = Pós graduação ou superior completo
- idade
- tempo_emprego 


As demais variáveis apresentam ou o Coef negativo ou P-value > 0.05

In [30]:
#3 Siga removendo as variáveis menos significantes, sempre que o p-value for menor que 5%. 
#Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

    #Levando em consideração variáveis que possuem P >0.05

    #Removendo a variável 'educacao', 'posse_de_veiculo'  'tipo_renda', 'tipo_residencia', 'qt_pessoas_residencia', ' qtd_filhos'

y_c, X_c = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_imovel)
                    + idade
                    + tempo_emprego
                    + 1''', df)

Modelo_C = sm.OLS(y_c,X_c).fit()
Modelo_C.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     907.7
Date:                Mon, 13 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:46:39   Log-Likelihood:                -13335.
No. Observations:               12466   AIC:                         2.668e+04
Df Residuals:                   12461   BIC:                         2.672e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      7.3732      0.030    243.856      0.000       7.314       7.432
C(sexo)[T.M]                   0.4742      0.013     35.424      0.000       0.448       0.500
C(posse_de_imovel)[T.True]     0.1018      0.013      7.649      0.000       0.076       0.128
idade                          0.0043      0.001      5.958      0.000       0.003       0.006
tempo_emprego                  0.0479      0.001     48.283      0.000       0.046       0.050
==============================================================================
Omnibus:                        1.296   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.523   Jarque-Bera (JB):                1.272
Skew:                           0.011   Prob(JB):                        0.529
Kurtosis:                       3.045   Cond. No.                         207.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""




**Modelo inicial**
R² = 0.242

R² - ajustado = 0.240

AIC = 2.646e+04

Modelo após retirar todas variáveis com P >0.05:

Sobraram 4 variáveis, todas com coef positivo e P <0.05 :

Dados do modelo:
R² = 0.226

R² - ajustado = 0.225

AIC =  	2.668e+04.

**Conclusão:** 
A retirada de diversas variáveis teve o objetivo de reduzir o overfitting. 
Apesar dos valores de R² e R² - ajustado diminuirem e o AIC ter aumentado, houveram impactos mínimos.
O modelo é portanto melhor que o inicial, porque nos leva a uma visão mais concreta possível sem levar a conclusões errôneas por variáveis que não eram estatisticamente relevantes.
